### 딥러닝 (3)

In [2]:
# nand or and조합없이 딥러닝으로 xor문제를 해결해보자

In [3]:
# xor은 0110 이 나와야함, 입력층 은닉층 출력층이 있는 딥러닝으로 풀어보자

In [4]:
# 입력층 노드는 2개 출력층노드 1개 은닉층노드는 적당히 6개로 했다.
# 딥러닝에서는 1개이상의 은닉층을 만들수있고, 노드도 임의로 만들수있다. 많으면 속도가 느리므로 적절히 하자.
# y의 loss가 최소가 될때까지 W,b를 업데이트함

In [5]:
import numpy as np

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [6]:
class LogicGate:
        
    def __init__(self, gate_name, xdata, tdata):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화 생성자 init
        self.__xdata = xdata.reshape(4,2)  # 4개의 입력데이터 x1, x2 에 대하여 batch 처리 행렬
        self.__tdata = tdata.reshape(4,1)  # 4개의 입력데이터 x1, x2 에 대한 각각의 계산 값 행렬
        
        # 2층 hidden layer unit : 6개 가정,  가중치 W2, 바이어스 b2 초기화
        self.__W2 = np.random.rand(2,6)  # weight, 2 X 6 matrix
        self.__b2 = np.random.rand(6)
        
        # 3층 output layer unit : 1 개 , 가중치 W3, 바이어스 b3 초기화
        self.__W3 = np.random.rand(6,1)
        self.__b3 = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.__learning_rate = 1e-2
    
        print(self.name + " object is created")
            
    def feed_forward(self):        # feed forward 를 통하여 손실함수(cross-entropy) 값 계산
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )    
    
    def loss_val(self):          # 외부 출력을 위한 손실함수(cross-entropy) 값 계산 
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력값
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )
    
    
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        for step in  range(10001):
            
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
    
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
        
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
    
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
    
            if (step % 400 == 0):
                print("step = ", step, "  , loss value = ", self.loss_val())
                
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.__W2) + self.__b2         # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

In [7]:
# AND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

and_obj = LogicGate("AND", xdata, tdata)

and_obj.train()

AND object is created
Initial loss value =  10.177319382593089
step =  0   , loss value =  9.87059461112306
step =  400   , loss value =  2.2536487710612025
step =  800   , loss value =  2.0784506441016712
step =  1200   , loss value =  1.787074275246396
step =  1600   , loss value =  1.4441642800346937
step =  2000   , loss value =  1.1067415917379653
step =  2400   , loss value =  0.8206177628603226
step =  2800   , loss value =  0.6058776360203502
step =  3200   , loss value =  0.45545975045931464
step =  3600   , loss value =  0.3519536144288472
step =  4000   , loss value =  0.27982243643434745
step =  4400   , loss value =  0.22828433858146582
step =  4800   , loss value =  0.19042165546224527
step =  5200   , loss value =  0.16185400630239621
step =  5600   , loss value =  0.13977256729001286
step =  6000   , loss value =  0.12233588702821309
step =  6400   , loss value =  0.10830627113484931
step =  6800   , loss value =  0.09683099471849525
step =  7200   , loss value =  0.087

In [8]:
# AND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(and_obj.predict(data))

(array([0.00013596]), 0)
(array([0.01229456]), 0)
(array([0.01253344]), 0)
(array([0.97554936]), 1)


In [9]:
# OR Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

or_obj = LogicGate("OR", xdata, tdata)

or_obj.train()

OR object is created
Initial loss value =  2.2600417939248
step =  0   , loss value =  2.2516608253850285
step =  400   , loss value =  1.9039635903694667
step =  800   , loss value =  1.653959115541854
step =  1200   , loss value =  1.3148275208093003
step =  1600   , loss value =  0.9649145285844649
step =  2000   , loss value =  0.685384232259387
step =  2400   , loss value =  0.49196337015460584
step =  2800   , loss value =  0.3643529229911088
step =  3200   , loss value =  0.2796695189709504
step =  3600   , loss value =  0.2218638686976874
step =  4000   , loss value =  0.18103144542153735
step =  4400   , loss value =  0.1512133502601806
step =  4800   , loss value =  0.12877754006551392
step =  5200   , loss value =  0.11144985992374404
step =  5600   , loss value =  0.09776204237450192
step =  6000   , loss value =  0.08673700979683817
step =  6400   , loss value =  0.07770593220139377
step =  6800   , loss value =  0.0701990533492466
step =  7200   , loss value =  0.06387869

In [10]:
# NAND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

nand_obj = LogicGate("NAND", xdata, tdata)

nand_obj.train()

NAND object is created
Initial loss value =  3.0077250861177482
step =  0   , loss value =  2.9837528776989153
step =  400   , loss value =  2.2843007981205954
step =  800   , loss value =  2.1365189636080677
step =  1200   , loss value =  1.8635321150054232
step =  1600   , loss value =  1.4920974586047309
step =  2000   , loss value =  1.1306569390867809
step =  2400   , loss value =  0.828713701568043
step =  2800   , loss value =  0.6051276148792346
step =  3200   , loss value =  0.4502826256430053
step =  3600   , loss value =  0.34439641076021765
step =  4000   , loss value =  0.2710355293345995
step =  4400   , loss value =  0.21899800885754195
step =  4800   , loss value =  0.18108734061075077
step =  5200   , loss value =  0.1527333753340328
step =  5600   , loss value =  0.13100550385126242
step =  6000   , loss value =  0.11398790816022344
step =  6400   , loss value =  0.10039914283650042
step =  6800   , loss value =  0.08936163834583474
step =  7200   , loss value =  0.08

In [11]:
# XOR Gate 객체 생성

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])


xor_obj = LogicGate("XOR", xdata, tdata)

xor_obj.train()

XOR object is created
Initial loss value =  4.136536415712109
step =  0   , loss value =  4.058382299522533
step =  400   , loss value =  2.7799199826068826
step =  800   , loss value =  2.7751023883222223
step =  1200   , loss value =  2.7703233546701655
step =  1600   , loss value =  2.7650666599484173
step =  2000   , loss value =  2.7587201963772645
step =  2400   , loss value =  2.7504751378107613
step =  2800   , loss value =  2.739209133941519
step =  3200   , loss value =  2.723354191348516
step =  3600   , loss value =  2.7007599385622925
step =  4000   , loss value =  2.668592669351268
step =  4400   , loss value =  2.623432534351982
step =  4800   , loss value =  2.5619878029771552
step =  5200   , loss value =  2.482793566639679
step =  5600   , loss value =  2.3879623385351616
step =  6000   , loss value =  2.282680513529932
step =  6400   , loss value =  2.1721796760240357
step =  6800   , loss value =  2.0589735685197055
step =  7200   , loss value =  1.9421758075418087


In [12]:
test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(xor_obj.predict(data))

(array([0.11925183]), 0)
(array([0.81110114]), 1)
(array([0.79659877]), 1)
(array([0.27713434]), 0)
